# microsoftmt
> Library to use Microsoft's machine translation services

In [ ]:
#| default_exp microsoftmt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests, uuid
import os
from langcodes import *

class microsofttranslate:
    """
    Class to get translations from the Microsoft Translator API
    """
    def __init__(self):
        """Constructor of microsofttranslate class"""
        self._subscription_key = os.getenv('MS_SUBSCRIPTION_KEY')
        self._region = os.getenv('MS_REGION')
        self._endpoint = "https://api.cognitive.microsofttranslator.com"
        self._languages_cache = []
        
    def check_langpair(self, sourcelang, targetlang):
        """Function to verify if a language pair identified by language ids is supported"""
        path = '/languages'
        constructed_url = self._endpoint + path
        params = {
            'api-version': '3.0',
            'scope': 'translation'
        }
        
        supported_languages = self._languages_cache
        # Cached language array empty if not initialized yet
        if not supported_languages:
            # Note that we aren't capturing the ETag from response header for the current language list -
            # we are caching the list after the first call and assume that it doesn't change over the lifetime
            # of the class,  we don't persist the list and call the API again when the class is re-instantiated
            request = requests.get(constructed_url, params=params)
            response = request.json()
            supported_languages = response["translation"].keys()
            self._languages_cache = supported_languages
            
        language_pair_supported = False
        if tag_is_valid(sourcelang) and tag_is_valid(targetlang):
            if sourcelang in supported_languages and targetlang in supported_languages:
                language_pair_supported = True
        
        return language_pair_supported

    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language"""
        translated_text = ""
        if self.check_langpair(sourcelang,targetlang):
            # Add your subscription key and endpoint
            subscription_key = self._subscription_key
            endpoint = self._endpoint

            # Add your location, also known as region. The default is global.
            # This is required if using a Cognitive Services resource.
            location = self._region

            path = '/translate'

            params = {
                'api-version': '3.0',
                'from': sourcelang,
                'to': [targetlang]
            }
            constructed_url = endpoint + path

            headers = {
                'Ocp-Apim-Subscription-Key': self._subscription_key,
                'Ocp-Apim-Subscription-Region': location,
                'Content-type': 'application/json',
                'X-ClientTraceId': str(uuid.uuid4())
            }

            # You can pass more than one object in body.
            body = [{
                'text': text
            }]

            request = requests.post(constructed_url, params=params, headers=headers, json=body)
            response = request.json()
            translated_text = response[0]["translations"][0]["text"]

        return translated_text

In [ ]:
#| hide
msmt = microsofttranslate()
translation = msmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una macchina fantastica!"

assert True == msmt.check_langpair("en","it")
assert True == msmt.check_langpair("en","gu")

In [ ]:
show_doc(microsofttranslate.check_langpair)

In [ ]:
show_doc(microsofttranslate.translate_text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()